In [1]:
import torch 
import melt 
import lele
import numpy as np
from prettytable import PrettyTable

tensorflow_version: 1.14.0
torch_version: 1.2.0
Using TensorFlow backend.


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
dir = '/home/gezi/new/temp/feed/rank/zjx_data_2/'

In [8]:
names = ['IGNORE'] + [x.strip().split('\t')[0] for x in open(dir + '/feature_index').readlines()]

In [10]:
names[-1]

'long_term_txk\x07风水命理_龙脉'

In [11]:
fnames = ['IGNORE'] + [x.rstrip().split('\t')[0] for x in open(dir + '/feat_fields.txt').readlines()]

In [13]:
len(fnames)

95

In [ ]:
names[10:20]

In [14]:
def wide_importance(wide_weight, names, type='all', reverse=False, num=10):
    wide_weight = np.squeeze(wide_weight)
    if type == 'all':
        w = -wide_weight * wide_weight
    elif type == 'pos':
        w = -wide_weight
    else:
        w = wide_weight
    if reverse:
        w = -w
    idx = np.argsort(w)
    t = PrettyTable(['index', 'name', 'weight'])
    for i in range(num):
      if idx[i] <= len(names):
        t.add_row([i, names[idx[i] - 1], wide_weight[idx[i]]])
    print(t)

In [ ]:
model_path = '/home/gezi/new/temp/feed/rank/zjx_data_2/model/v9/'

In [ ]:
checkpoint = torch.load(model_path)
wide_weight = checkpoint['state_dict']['wide.emb.weight']
wide_weight = wide_weight.cpu().numpy()

In [ ]:
def emb_importance(deep_weight, names, reverse=False, num=10):
  w = [sum(x * x) for x in deep_weight]
  wide_importance(w, names, 'pos', reverse=reverse, num=num)

In [ ]:
deep_weight = checkpoint['state_dict']['deep.emb.weight'].cpu().numpy()

In [ ]:
checkpoint['state_dict'].keys()

In [ ]:
field_weight = checkpoint['state_dict']['deep.field_emb.weight'].cpu().numpy()

In [ ]:
emb_importance(deep_weight, names, True, 100)

In [ ]:
emb_importance(deep_weight, names, False, 100)

In [ ]:
emb_importance(field_weight, fnames, False, 100)

In [ ]:
len(field_weight)

In [ ]:
len(fnames)

In [ ]:
emb_importance(field_weight, fnames, True, 100)

In [ ]:
wide_importance(wide_weight, names, 'all')

In [ ]:
wide_importance(wide_weight, names, 'pos')

In [ ]:
wide_importance(wide_weight, names, 'neg')

In [ ]:
model_dir = '/home/gezi/new/temp/feed/rank/input/model/WideDeep.best.v0'

In [ ]:
from tensorflow.python import pywrap_tensorflow
model_path = melt.get_model_path(model_dir)
print(model_path)
reader = pywrap_tensorflow.NewCheckpointReader(model_path)

In [ ]:
checkpoint = reader.get_variable_to_shape_map()

In [ ]:
deep_weight =  reader.get_tensor('wide_deep/deep/embedding_1/embeddings')

In [ ]:
deep_importance(deep_weight, names)

In [ ]:
reader.get_tensor('wide_deep/dense_2/kernel')

In [ ]:
wide_weight = reader.get_tensor('wide_deep/wide/embedding/embeddings')

In [ ]:
wide_importance(wide_weight, names, 'all', True, 100)

In [ ]:
wide_importance(wide_weight, names, 'all', False)

In [ ]:
wide_importance(wide_weight, names, 'pos', False, 100)

In [ ]:
wide_importance(wide_weight, names, 'pos', True, 100)

In [ ]:
wide_importance(wide_weight, names, 'neg', 100)